In [49]:
import datetime
import pandas as pd
from datetime import time,date,timedelta
import requests
import matplotlib.pyplot as plt
import numpy as np
import ta

def test_live(sample_duration):
    def vn30f():
        return requests.get("https://services.entrade.com.vn/chart-api/chart?from=1651727820&resolution=1&symbol=VN30F1M&to=9999999999").json()
    vn30fm = pd.DataFrame(vn30f()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    s = pd.read_csv(r"C:\Users\FPTSHOP\Desktop\FP.d\data.csv")
    s['Date'] = pd.to_datetime(s['Date'])
    ohlc_dict = {                                                                                                             
        'Open': 'first',                                                                                                    
        'High': 'max',                                                                                                       
        'Low': 'min',                                                                                                        
        'Close': 'last',                                                                                                    
        'Volume': 'sum',}
    vn30fm = pd.DataFrame(vn30f()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    def process_data(input_df):
        vn30train = pd.DataFrame(input_df.resample(str(sample_duration)+'Min', on='Date', label='left').apply(ohlc_dict).dropna()).reset_index()#change s
        vn30train['Date'] = [str(i)[:16] for i in vn30train['Date']]
        return vn30train
    vn30f_base = pd.concat([process_data(vn30fm), process_data(s)]).sort_values('Date').drop_duplicates('Date').sort_values('Date')
    return vn30f_base

def test_livevn30(sample_duration):
    def vn30():                                     
        return requests.get("https://services.entrade.com.vn/chart-api/v2/ohlcs/index?from=0&resolution=1&symbol=VN30&to=9999999999").json()
    vn30fm = pd.DataFrame(vn30()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    s = pd.read_csv('C:/Users/ducbu/Downloads/DataMinute/VN30.csv')
    s['Date'] = pd.to_datetime(s['Date']) + timedelta(hours =7)

    ohlc_dict = {                                                                                                             
        'Open': 'first',                                                                                                    
        'High': 'max',                                                                                                       
        'Low': 'min',                                                         
        'Close': 'last',                                                                                                    
        'Volume': 'sum',}
    vn30fm = pd.DataFrame(vn30()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    def process_data(input_df):
        vn30train = pd.DataFrame(input_df.resample(str(sample_duration)+'Min', on='Date', label='left').apply(ohlc_dict).dropna()).reset_index()#change s
        vn30train['Date'] = [str(i)[:16] for i in vn30train['Date']]
        return vn30train
    vn30f_base = pd.concat([process_data(vn30fm), process_data(s)]).sort_values('Date').drop_duplicates('Date').sort_values('Date')
    return vn30f_base    

def get_stock_data(symbol, sample_duration):
    ticker = symbol
    link = "https://services.entrade.com.vn/chart-api/v2/ohlcs/stock?from=0&resolution=1&symbol={ticker}&to=2654048800".format(ticker=ticker)
    f = requests.get(link)
    data = pd.DataFrame(f.json()).iloc[:,:6]
    data['t'] = data['t'].astype(int).apply(lambda x: datetime.datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    data.columns = ['Date','Open','High','Low','Close','Volume']
    s = pd.read_csv('C:/Users/ducbu/Downloads/DataMinute/' + symbol + '.csv')
    s['Date'] = pd.to_datetime(s['Date']) +timedelta(hours =7)
    ohlc_dict = {                                                                                                             
        'Open': 'first',                                                                                                    
        'High': 'max',                                                                                                       
        'Low': 'min',                                                         
        'Close': 'last',                                                                                                    
        'Volume': 'sum',}
    data.columns = ['Date','Open','High','Low','Close','Volume']
    def process_data(input_df):
        vn30train = pd.DataFrame(input_df.resample(str(sample_duration)+'Min', on='Date', label='left').apply(ohlc_dict).dropna()).reset_index()#change s
        vn30train['Date'] = [str(i)[:16] for i in vn30train['Date']]
        return vn30train
    vn30f_base = pd.concat([process_data(data), process_data(s)]).sort_values('Date').drop_duplicates('Date').sort_values('Date')
    return vn30f_base

In [50]:
data= test_live(5)

In [51]:
data.set_index('Date', inplace=True)
data.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20,942.6,943.7,942.4,943.7,1765


In [58]:
data["F1"] = [(data["High"][i]+data["Low"][i]-data["Close"][i]*2)*data["Volume"][i]/2 for i in range(len(data))]
data["difference"] = data["Close"].rolling(2).apply(lambda x: x.iloc[1] - x.iloc[0])
data["Target"] = data["Close"].rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])
data.head()

,Open,High,Low,Close,Volume,F1,difference,Target
Date,,,,,,,,
2018-08-13 09:00,943.5,943.6,942.9,943.1,1812,271.80,NaN,NaN
2018-08-13 09:05,943.1,943.5,942.9,943.3,1323,-132.30,0.2,1.0
2018-08-13 09:10,943.2,943.3,942.6,943.1,1207,-181.05,-0.2,0.0
2018-08-13 09:15,943.1,943.1,942.3,942.6,1196,119.60,-0.5,0.0
2018-08-13 09:20,942.6,943.7,942.4,943.7,1765,-1147.25,1.1,1.0


In [59]:
data.to_csv('datavn30fm.csv')